In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [2]:
# the measurement station codes (35 stations)
# more detailed information on the stations can be found in metadata/measurement_sites_info.txt
stations = ['ABZ', 'ALE', 'AMA', 'AMM', 'ASP', 'BEI', 'BOT', 'BSL', 'DEL', 'EGB',
            'FKL', 'HAD', 'HEL', 'HPB', 'HRW', 'HYY', 'KCE', 'KPZ', 'MAR', 'MHD', 
            'MLP', 'MUK', 'NAN', 'NEU', 'POV', 'SAO', 'SCH', 'SGP', 'UAE', 'PRL',
            'VAR', 'VHL', 'VIE', 'WAL', 'ZOT']

In [56]:
def combine_datasets(folder):
    '''
    Concatenates the individual datasets for all measurement stations in a specified folder located in "data/".
    
    Parameters:
    folder (str): Name of folder located in the "data/" folder of the repository

    Returns:
    pd.DataFrame: data frame of the concatenated data sets in the folder
    '''
    
    full_df = []

    for s in stations:
        df = pd.read_csv('data/'+folder+'/'+s+'.csv')
        df['station'] = s
        full_df.append(df)

    full_df = pd.concat(full_df)
    full_df = full_df.reset_index(drop=True)
    full_df['id'] = [full_df.station[i] + '-' + str(full_df.date[i]) for i in range(full_df.shape[0])]
    
    return full_df

In [65]:
# loading the aerosol mixing ratio data and saving it as a separate csv file
# a full description of all variable names can be found in metadata/variable_names.txt
aerosols_df = combine_datasets('aerosols')
aerosols_df = aerosols_df[['id', 'date', 'station', 'latitude', 'longitude', 'aermr01', 'aermr02', 
                           'aermr03', 'aermr04', 'aermr05', 'aermr06', 'aermr07', 
                           'aermr08', 'aermr09', 'aermr10', 'aermr11']]
aerosols_df.to_csv('data/aerosols_data.csv', index=False)

In [75]:
# loading the atmospheric data, calculating the relative humidity, and saving it as a separate csv file
atmospheric_df = combine_datasets('atmospheric')
atmospheric_df = atmospheric_df[['id', 'd2m', 't2m']]
td = atmospheric_df.d2m - 273.15
t = atmospheric_df.t2m - 273.15
atmospheric_df['rh'] = 100*(np.exp((17.625*td) / (243.04+td)) / np.exp((17.625*t) / (243.04+t)))
atmospheric_df.to_csv('data/atmospheric_data.csv', index=False)

In [67]:
# loading the boundary layer height data and saving it as a separate csv file
boundary_layer_height_df = combine_datasets('boundary_layer_height')
boundary_layer_height_df = boundary_layer_height_df[['id', 'blh']]
boundary_layer_height_df.to_csv('data/boundary_layer_height_data.csv', index=False)

In [68]:
# loading the gas data and saving it as a separate csv file
gases_df = combine_datasets('gases')
gases_df = gases_df[['id', 'co', 'c5h8', 'no2', 'no', 'so2']]
gases_df.to_csv('data/gases_data.csv', index=False)

In [69]:
# loading the slow access data and saving it as a separate csv file
slow_access_df = combine_datasets('slow_access')
slow_access_df = slow_access_df[['id', 'nh3', 'crwc', 'c10h16']]
slow_access_df.to_csv('data/slow_access_data.csv', index=False)

In [70]:
# loading the wind data, calculating the wind speed, and saving it as a separate csv file
wind_df = combine_datasets('wind')
wind_df['wind_speed'] = np.sqrt(wind_df.u**2 + wind_df.v**2)
wind_df = wind_df[['id', 'wind_speed']]
wind_df.to_csv('data/wind.csv', index=False)

In [83]:
# combining all the data sets into one dataframe
data = aerosols_df.merge(atmospheric_df, on='id')
data = data.merge(boundary_layer_height_df, on='id')
data = data.merge(gases_df, on='id')
data = data.merge(slow_access_df, on='id')
data = data.merge(wind_df, on='id')

# reordering the columns
data = data[['station', 'date', 'latitude', 'longitude', 
             'aermr01', 'aermr02', 'aermr03', 'aermr04', 'aermr05', 'aermr06', 'aermr07', 
             'aermr08', 'aermr09', 'aermr10', 'aermr11', 'co', 'c5h8', 'c10h16', 'nh3', 
             'no', 'no2', 'so2', 'd2m', 't2m', 'crwc', 'blh', 'rh', 'wind_speed']]

# saving the full dataset
data.date = pd.to_datetime(data.date)
data.dropna(inplace=True)
data.to_csv('data/covariates.csv', index=False)
data.head()

,station,date,latitude,longitude,aermr01,aermr02,aermr03,aermr04,aermr05,aermr06,...,nh3,no,no2,so2,d2m,t2m,crwc,blh,rh,wind_speed
0,ABZ,2012-01-01 00:00:00,50.57,12.99,9.728344e-12,8.303498e-10,1.977892e-10,4.955472e-13,8.809125e-13,0.000000e+00,...,4.947672e-10,3.451908e-10,1.688608e-08,4.579990e-09,274.34976,274.74243,6.662354e-07,226.21110,97.224089,2.388552
1,ABZ,2012-01-01 03:00:00,50.57,12.99,1.149119e-11,9.807117e-10,3.608436e-10,8.954750e-13,1.505326e-12,0.000000e+00,...,4.704389e-10,5.595195e-10,1.851567e-08,3.697133e-09,275.56702,275.84317,1.356097e-06,227.60513,98.057947,2.525840
2,ABZ,2012-01-01 06:00:00,50.57,12.99,1.498656e-11,1.278999e-09,5.850223e-10,1.004596e-12,1.536192e-12,0.000000e+00,...,3.911320e-10,7.258628e-10,1.803737e-08,1.453171e-09,276.51514,276.86853,3.219190e-06,296.77762,97.541081,2.777447
3,ABZ,2012-01-01 09:00:00,50.57,12.99,1.326326e-11,1.131177e-09,6.964777e-10,6.364222e-13,8.816045e-13,0.000000e+00,...,3.703658e-10,1.102613e-09,1.353255e-08,1.221443e-09,277.77420,278.42493,1.323438e-07,641.16990,95.568645,3.490278
4,ABZ,2012-01-01 12:00:00,50.57,12.99,7.933063e-12,6.726889e-10,3.573501e-10,2.600534e-13,4.015943e-13,2.105410e-13,...,6.122887e-10,7.667192e-10,8.655749e-09,1.667808e-09,279.82742,280.83896,3.709535e-07,586.82715,93.313550,3.836724


In [111]:
# loading the n100 data and saving it as a separate csv file
n100_df = []
for s in stations:
    df = pd.read_table(f'data/N100_proxy/{s}_N100.dat', sep='\s+', 
                       names=['year', 'month', 'day', 'hour', 'minute', 'n100'])
    
    # AMA and UAE are in UTC, so the times of the measurements need to be adjusted to local time
    if (s == 'AMA'):
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.date -= timedelta(hours=4)     
    elif (s == 'UAE'):
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        df.date += timedelta(hours=4)
        
    # the year for FKL is multiplied by 2 due to some bug, so needs to be corrected
    elif (s == 'FKL'):
        df.year /= 2
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])  
    # removing outliers for VHL
    elif (s == 'VHL'):
        for i, row in df.iterrows():
            if row.n100 > 10000:
                df.drop(index=i, inplace=True)
    else:
        df['date'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
        
    df.drop(columns=['year', 'month', 'day', 'hour', 'minute'], inplace=True)
    
    df['station'] = s
    n100_df.append(df)

n100_df = pd.concat(n100_df)
n100_df = n100_df.reset_index(drop=True)
n100_df = n100_df.drop_duplicates()
n100_df.loc[n100_df['n100'] <= 0, 'n100'] = np.nan
# n100_df['id'] = [n100_df.station[i] + '-' + str(n100_df.date[i]) for i in range(n100_df.shape[0])]
n100_df = n100_df[['station', 'date', 'n100']]
n100_df.to_csv('data/n100_granular_data.csv', index=False)

In [4]:
n100_df = pd.read_csv('data/n100_granular_data.csv')

In [5]:
n100_df.head()

,station,date,n100
0,ABZ,2012-01-26 17:30:00,2967.4
1,ABZ,2012-01-26 18:29:00,2756.7
2,ABZ,2012-01-26 19:30:00,3260.1
3,ABZ,2012-01-26 20:30:00,3013.8
4,ABZ,2012-01-26 21:29:00,2525.4


In [18]:
def interpolate_with_time_limit(df, column, time_limit):
    jd_max_gap_fill = time_limit / (60*24)
    
    # calculate the value gap
    df['ffill'] = df[column].ffill()
    df['value_gap'] = df[column].bfill() - df[column].ffill()

    # get the Julian date for the entry
    df['jd'] = df.index.to_julian_date()

    # calculate the time gap
    df['jd_nan'] = np.where(~df[column].isna(), df['jd'], np.nan)
    df['jd_gap'] = df['jd_nan'].bfill() - df['jd_nan'].ffill()
    
    # time-wise, calculate how far into the value gap we are
    df['jd_start'] = df['jd_nan'].ffill() 
    df['jd_prp'] = np.where(df['jd_gap'] != 0, (df['jd'] - df['jd_start']) / df['jd_gap'], 0)
    
    # calculate time-interpolated values
    filled_value = np.where(df['jd_gap'] <= jd_max_gap_fill, df['ffill'] + df['value_gap'] * df['jd_prp'], np.nan) 

    return filled_value

In [33]:
dfs = []

for station in stations:
    df_station = n100_df[n100_df['station'] == station].copy()

    # convert date column to datetime and set as index
    df_station['date'] = pd.to_datetime(df_station['date'])
    df_station.set_index('date', inplace=True)
    
    # remove duplicate measurements
    df_station = df_station[~df_station.index.duplicated()]
    
    # compute the difference between consecutive rows
    differences = df_station['n100'].diff()
    # filter out rows where the difference is zero
    df_station = df_station[differences != 0]

    # calculate time diffs between rows in minutes
    time_diffs = df_station.index.to_series().diff().dt.total_seconds().div(60).dropna()
    # compute the mode
    mode_diff = time_diffs.mode()[0] # mode() returns a Series, get the first value
    # create a new dataframe by resampling the original
    df_resampled = df_station.resample(f'{int(mode_diff)}T').asfreq()
    # merge the original dataframe with the resampled one
    df_station = pd.concat([df_station, df_resampled]).sort_index()
    
    # reset index
    df_station.reset_index(inplace=True)
    df_station.rename(columns={'index': 'date'}, inplace=True)
    # drop duplicate rows
    df_station.drop_duplicates(inplace=True)
    # set index back to date
    df_station.set_index('date', inplace=True)

    # calculate the time difference to the previous row in minutes
    time_diff_prev_row = df_station.index.to_series().diff().dt.total_seconds().div(60).abs()
    # calculate the time difference to the next row in minutes
    time_diff_next_row = df_station.index.to_series().diff(-1).dt.total_seconds().div(60).abs()
    # create a mask where 'n100' is NaN and the difference between current row and previous/next row is less than mode_diff - 1
    mask = (df_station['n100'].isna()) & ((time_diff_prev_row < mode_diff - 1) | (time_diff_next_row < mode_diff - 1))
    # drop these rows
    df_station = df_station.loc[~mask]
    
    # set station again
    df_station['station'] = station
    
    # drop leading NaNs
    df_station = df_station.loc[pd.notna(df_station['n100']).idxmax():]
    # drop trailing NaNs
    df_station = df_station.loc[:df_station['n100'].notna()[::-1].idxmax()]
    
    # interpolate sequences 1 hour or shorter
    nans_before = df_station.n100.isna().sum()
    df_station['n100_filled'] = interpolate_with_time_limit(df_station.copy(), column='n100', time_limit=60)
    df_station['interpolated'] = (df_station['n100'] != df_station['n100_filled']) & ~(df_station['n100'].isna() & df_station['n100_filled'].isna())
    df_station['n100'] = df_station['n100_filled']
    df_station.drop(columns=['n100_filled'], inplace=True)
    nans_after = df_station.n100.isna().sum()
    
    # append processed dataframe to the list
    dfs.append(df_station)
    
    print(f'{station} mode diff: {int(mode_diff)} min \t gaps: {nans_before} -> {nans_after}')

# concatenate all dataframes
df_final = pd.concat(dfs)

df_final.to_csv('data/n100_final_data.csv')

ABZ mode diff: 59 min 	 gaps: 14154 -> 14154
ALE mode diff: 14 min 	 gaps: 12085 -> 11909
AMA mode diff: 5 min 	 gaps: 109386 -> 109217
AMM mode diff: 5 min 	 gaps: 7399 -> 7306
ASP mode diff: 59 min 	 gaps: 33341 -> 33341
BEI mode diff: 8 min 	 gaps: 33828 -> 33671
BOT mode diff: 9 min 	 gaps: 23551 -> 23197
BSL mode diff: 60 min 	 gaps: 8017 -> 8017
DEL mode diff: 60 min 	 gaps: 5133 -> 5133
EGB mode diff: 15 min 	 gaps: 4308 -> 4153
FKL mode diff: 5 min 	 gaps: 31054 -> 30944
HAD mode diff: 5 min 	 gaps: 85126 -> 84848
HEL mode diff: 10 min 	 gaps: 19966 -> 18036
HPB mode diff: 60 min 	 gaps: 9902 -> 9902
HRW mode diff: 60 min 	 gaps: 3503 -> 3503
HYY mode diff: 10 min 	 gaps: 17663 -> 13160
KCE mode diff: 60 min 	 gaps: 8430 -> 8430
KPZ mode diff: 9 min 	 gaps: 167412 -> 167191
MAR mode diff: 9 min 	 gaps: 27342 -> 27144
MHD mode diff: 59 min 	 gaps: 8536 -> 8536
MLP mode diff: 60 min 	 gaps: 12405 -> 12405
MUK mode diff: 60 min 	 gaps: 11068 -> 11068
NAN mode diff: 9 min 	 gaps: 2

In [45]:
df_final = pd.read_csv('data/n100_final_data.csv')
df_final['date'] = pd.to_datetime(df_final['date'])
df_final.set_index('date', inplace=True)
df_final.head()

,station,n100,interpolated
date,,,
2012-01-26 17:30:00,ABZ,2967.4,False
2012-01-26 18:29:00,ABZ,2756.7,False
2012-01-26 19:30:00,ABZ,3260.1,False
2012-01-26 20:30:00,ABZ,3013.8,False
2012-01-26 21:29:00,ABZ,2525.4,False


In [46]:
print(f'Interpolated {df_final.interpolated.sum()} / {df_final.shape[0]}')

Interpolated 35629 / 5087274


In [47]:
def print_gaps(df):
    stations = df['station'].unique()
    for station in stations:
        df_station = df[df['station'] == station]
        gaps = df_station['n100'].isna()
        print(f'Station: {station}')
        print(f'Number of NaNs: {gaps.sum()}')
        gap_lengths = gaps.diff().ne(0).cumsum()[gaps].value_counts().sort_index()
        print('Number of gaps:', len(gap_lengths))
        print('Min:', gap_lengths.min(), 'Max:', gap_lengths.max())
        print('Mean:', round(gap_lengths.mean(), 2), 'Std:', round(gap_lengths.std(), 2))
        print("-----------------------------")

print_gaps(df_final)

Station: ABZ
Number of NaNs: 14154
Number of gaps: 497
Min: 1 Max: 4614
Mean: 28.48 Std: 249.76
-----------------------------
Station: ALE
Number of NaNs: 11909
Number of gaps: 553
Min: 2 Max: 1371
Mean: 21.54 Std: 67.72
-----------------------------
Station: AMA
Number of NaNs: 109217
Number of gaps: 122
Min: 10 Max: 31074
Mean: 895.22 Std: 3585.44
-----------------------------
Station: AMM
Number of NaNs: 7306
Number of gaps: 27
Min: 11 Max: 2701
Mean: 270.59 Std: 576.26
-----------------------------
Station: ASP
Number of NaNs: 33341
Number of gaps: 55
Min: 1 Max: 29678
Mean: 606.2 Std: 3997.01
-----------------------------
Station: BEI
Number of NaNs: 33671
Number of gaps: 100
Min: 3 Max: 4866
Mean: 336.71 Std: 647.89
-----------------------------
Station: BOT
Number of NaNs: 23197
Number of gaps: 124
Min: 5 Max: 2718
Mean: 187.07 Std: 506.68
-----------------------------
Station: BSL
Number of NaNs: 8017
Number of gaps: 24
Min: 1 Max: 4416
Mean: 334.04 Std: 970.63
----------------